# Quantum Classifier

Está dividido em 4 grandes blocos:

## 1-Função de perda
## 2-Codificação dos dados clássicos
## 3-Tipo da rede neural
## 4-Modelo de otimização

# Hello World!

Template que atende 90% dos casos de uso. Está divido em 4 passos.

1-Carregamento dos dados para treinamento e teste:

In [1]:
from QuantumClassifier import *
from sklearn.model_selection import train_test_split

In [2]:
X, y = load_iris([0,1])

seed = 1977
training_inputs, test_inputs, \
training_labels, test_labels =  train_test_split(X, y, test_size = 0.66, random_state = seed)

2-Hiperparâmetros do modelo:

In [3]:
lss = SquaredLoss()            # loss function type.                         [1-AbsoluteLoss, 2-CrossEntropyLoss, 3-SquaredLoss]
enc = IBMQubitEncoding()       # encode classical data vector.               [1-IBMQubitEncoding, 2-IBMAmplitudeEncoding]
qnn = IBM_TTN_Statevector(enc) # type / implementation of the neural network.[1-IBM_TTN_Statevector, 2-IBM_MERA_Statevector, 3-IBM_TTN_Simulator, 4-IBM_MERA_Simulator, 5-IBM_Bell_Statevector, 6-IBM_Bell_Simulator]
mdl = Adam(qnn, lss, a=0.25, h=0.001, ct=0.025, it=25) # optimization model. [1-GradientDescent, 2-Adam]

3-Ajuste do modelo:

In [4]:
params, t, c = mdl.fit(training_inputs, training_labels) # return model parameters for a given training dataset.
print("Custo final depois da iteração %i: %f" %(t, c))

Parametros iniciais:
 2.166072250493227 2.2348796733443583 -1.5295659618414725 -0.04560961768358718 0.6629808141937388 0.02760938730712592 0.51365041415082
Custo inicial: 0.392568
Custo depois da iteração 5: 0.032532
Custo final depois da iteração 7: 0.022134


4-Predição:

In [5]:
prediction = qnn.predict(test_inputs, params) # return predictions for a given model and test dataset.
print('Acurácia     : %.4f'%(100 - np.mean(np.abs(prediction - test_labels)) * 100) + '%')

Acurácia     : 98.4848%


# Variação

Demonstração de como usar o classificador para otimizar um circuito quântico.

Para fazer a otimização precisamos escrever o circuito utilzando parâmetros ajustáveis, como o mostrado abaixo.

<img src="https://raw.githubusercontent.com/israelferrazaraujo/comp-quantica/patch-3/projetos/2019-01/files/Bell_State_circuit.png" style="width:50%">

Esse é um circuito que gera os estados de Bell. Os dois primeiros $R_y$ codificam a entrada, que pode ser 0 ou 1 para cada qubit. Neste exemplo específico, $q_0=|0>$ e $q_1=|1>$. Os dois últimos $R_y$ são equivalentes ao Hadamard, que pode ser decomposto como $H=XY^{1/2}$. Essa escolha não é única, existem outras maneiras de codificar os dados e, também, construir o Hadamard (comumente com uma fase global envolvida).

No exemplo usamos o computador "ibmq_16_melbourne", escolhido por ser o mais ruidoso.

In [6]:
X2, y2 = load_bell_states()

seed = 1977
training_inputs2, test_inputs2, \
training_labels2, test_labels2 =  train_test_split(X2, y2, test_size = 0.66, random_state = seed)

In [7]:
lss = SquaredLoss()            # loss function type.
enc = IBMQubitEncoding()       # encode classical data vector.
qnn = IBM_Bell_Simulator(enc,'ibmq_16_melbourne')  # type / implementation of the neural network.
mdl = Adam(qnn, lss, a=0.1, h=0.01, ct=0.0, it=100, adnit=10) # optimization model.

In [ ]:
prediction = qnn.predict(test_inputs2, [0]*qnn.params_vector_size(2)) # Here the circuit parameters are not changed.
print('Custo antes da otimização    : %f' %mdl.cost(training_inputs2, training_labels2, [0]*qnn.params_vector_size(2)))
print('Acurácia antes da otimização : %.4f'%(100 - np.mean(np.abs(prediction - test_labels2)) * 100) + '%')

params, t, c = mdl.fit(training_inputs2, training_labels2) # return model parameters for a given training dataset.
prediction = qnn.predict(test_inputs2, params) # return predictions for a given model and test dataset.
print ("Custo final depois da iteração %i: %f" %(t, c))
print('Acurácia após a otimização   : %.4f'%(100 - np.mean(np.abs(prediction - test_labels2)) * 100) + '%')

Custo antes da otimização    : 0.029210

Acurácia antes da otimização : 100.0000%

Parametros iniciais:

 2.9052288337392485 -1.325529482232365 2.069230892508073 -3.013069495602396
 
Custo inicial: 0.176921

Custo depois da iteração 5: 0.078836

Custo depois da iteração 10: 0.039402

Custo depois da iteração 15: 0.029964

Custo depois da iteração 20: 0.031750

Custo depois da iteração 25: 0.034099

Custo final depois da iteração 17: 0.029781

Acurácia após a otimização   : 100.0000%

# Simulações

<img src="https://raw.githubusercontent.com/israelferrazaraujo/comp-quantica/patch-3/projetos/2019-01/files/Simulations.png">

In [ ]:
lss = SquaredLoss()            # loss function type.
enc = IBMQubitEncoding()       # encode classical data vector.
qnn = IBM_TTN_Simulator(enc, 'ibmq_london')           # type / implementation of the neural network.
mdl = Adam(qnn, lss, a=0.25, h=0.01, ct=0.025, it=30) # optimization model.

params, t, c = mdl.fit(training_inputs, training_labels) # return model parameters for a given training dataset.

prediction = qnn.predict(test_inputs, params) # return predictions for a given model and test dataset.

print('Custo final  :', c)
print('Acurácia     : %.4f'%(100 - np.mean(np.abs(prediction - test_labels)) * 100) + '%')

Parametros iniciais:

-0.10480255  2.99346817 -1.94746787  1.74899496  1.14576659 -2.39546485  2.16811462

Custo inicial: 0.138497

Custo final  : 0.020219179882722742

Acurácia     : 100.0000%

# Overview

<img src="https://raw.githubusercontent.com/israelferrazaraujo/comp-quantica/patch-3/projetos/2019-01/files/ClassDiagram.png" style="width:90%">

# Hipótese
## Codificação Automática de Dados Clássicos em Amplitudes de Estados de Emaranhados

A hipótese é que os modelos de redes tensoriais, implementadas em circuitos quânticos, permitem que dados clássicos sejam compactados e codificados em amplitudes de estados emaranhados de maneira automática, bastando haver um prévio treinamento dos parâmetros da rede.

Para ver como isso pode ser feitos, observemos os seguintes circuitos TTN, usando respectivamente Qubit Encoding e Amplitude Encoding:

<img src="https://raw.githubusercontent.com/israelferrazaraujo/comp-quantica/patch-3/projetos/2019-01/files/TTN_Qubit.png" style="width:75%">

<img src="https://raw.githubusercontent.com/israelferrazaraujo/comp-quantica/patch-3/projetos/2019-01/files/TTN_Amplitude.png" style="width:75%">

Sabemos, a partir dos testes previamente efetuados, que as redes TTN conseguem predizer corretamente quando os dados clássicos são codificados nas amplitudes de estados emaranhado. Esses testes utilizaram a função "initialize" do Qiskit.

Observando os circuitos acima e interpretando a primeira parte do primeiro circuito e o initialize do segundo como caixas pretas, percebemos que o restante do circuito é igual.

<img src="https://raw.githubusercontent.com/israelferrazaraujo/comp-quantica/patch-3/projetos/2019-01/files/TTN_Qubit_blackbox.png" style="width:75%">

<img src="https://raw.githubusercontent.com/israelferrazaraujo/comp-quantica/patch-3/projetos/2019-01/files/TTN_Amplitude_blackbox.png" style="width:75%">

Sendo os resultados finais iguais, podemos inferir que as caixas pretas são equivalentes. Ou seja, que a primeira parte do primeiro circuito está, na verdade, fazendo o mesmo que o "initialize", codificando os dados clássicos nas amplitiudes de um estado emaranhado!